In [ ]:
# import opencv-python as cv2 
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
whT = 320
confThreshold = 0.5 #detection threshold
nmsThreshold = 0.3 #non max suppression threshold.

In [ ]:
#read the classes in the file: 
classesFile = 'coco.names'
classNames = []
with open(classesFile,'rt') as f:
    classNames = f.read().rstrip('n').split('n')
# print(classNames)
print(len(classNames)) #28

28


In [ ]:
modelConfiguration = 'yolov3.cfg'
modelWeights = 'yolov3.weights'
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)

error: OpenCV(4.7.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'


In [ ]:
#create a function to find the images and assign the bounding box thresholds 
def findObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]* wT), int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))


    #print(len(bbox))
    indices = cv2.dnn.NMSBoxes(bbox, confs,confThreshold,nmsThreshold)

    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x,y),(x+w,y+h),(255,0,255),2)
        cv2.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                    (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(255,0,255),2)


In [ ]:
#for testing the setup, we'll write some code: 
while True:
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1/255,(whT,whT),[0,0,0],crop=False)
    net.setInput(blob)

    layerNames = net.getLayerNames()
    outputNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]

    outputs = net.forward(outputNames)
    findObjects(outputs,img)

    cv2.imshow('Image', img)
    cv2.waitKey(1)

<h1>YOLO Class detection

In [ ]:
#Load YOLO 
import numpy as np 
import cv2
net = cv2.dnn.readNet("/kaggle/input/yolo-coco-data/yolov3.weights", "/kaggle/input/yolo-coco-data/yolov3.cfg")
classes = []
with open("/kaggle/input/yolo-coco-data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
# print(len(classes))
layer_names = net.getLayerNames() 
output_layers = [layer_names[i -1] for i in net.getUnconnectedOutLayers()]
# colors = np.random.uniform(0, 255, size=(len(classes), 3))
# print(output_layers)

In [ ]:
img = cv2.imread("/kaggle/input/jpg-file/yolo_sample.jpg")
height, width, channels = img.shape

In [ ]:
cv2.imshow("Image",img) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [ ]:
from PIL import Image
img = cv2.imread('/kaggle/input/jpg-file/yolo_sample.jpg')
# Image.fromarray(img).show()
Image.fromarray(img, 'RGB').show()
